<a href="https://colab.research.google.com/github/daniel-sjkdm/PandaCovid/blob/master/PandaCovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid 19 analisys 

This analisys is performed using the pandas library.



## About the dataset

The data set has a .csv file for each day since 22 January 2020 to the 16 September 2020. The headers are:



## Sources

[Novel Coronavirus Dataset](https://www.kaggle.com/anjanatiha/corona-virus-time-series-dataset?)

In [41]:
!pip install tabulate -q  

In [94]:
from google.colab import drive
from pprint import pprint
import pandas as pd
import numpy as np
import tabulate
import plotly
import os
import re

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


### Reading files and creating the dataframe


In [12]:
# The files are stored as csv files for each day with the format dd-mm-yyyy.csv
# Here, I'll read the files one by one and concatenate them in a single dataframe
# Done in order with sorted function (by date)
# The empty fields are assigned a NaN value by default. A 0 constant is assigned
# when reading and writing the csv files with na_values and na_rep parameters
# respectively

dataset_path = "/content/drive/My Drive/Datasets/Covid/csse_covid_19_daily_reports"

if not os.path.exists(dataset_path + "/" + "total_cases.csv"):
  df = pd.DataFrame()
  for data_file in sorted(os.listdir(dataset_path)):
    if re.fullmatch(".*\.csv", data_file):
      df = pd.concat([df, pd.read_csv(dataset_path + "/" + data_file, na_values=None)], axis=0)
  df.to_csv("/content/drive/My Drive/Datasets/Covid/csse_covid_19_daily_reports/total_cases.csv", sep=",", na_rep=None)
else:
  df = pd.read_csv(dataset_path + "/" + "total_cases.csv")

df.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# The columns of the data are the following
# In this analisys I'm only interested in some of them 
# so the other ones will be dropped

print("Original keys")
pprint(list(df.keys()))

labels = [
          "Latitude",
          "Longitude",
          "FIPS",
          "Admin2",
          "Lat",
          "Long_",
          "Combined_Key",
          "Incidence_Rate",
          "Case-Fatality_Ration"
]



Original keys
['Unnamed: 0',
 'Province/State',
 'Country/Region',
 'Last Update',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Latitude',
 'Longitude',
 'FIPS',
 'Admin2',
 'Province_State',
 'Country_Region',
 'Last_Update',
 'Lat',
 'Long_',
 'Active',
 'Combined_Key',
 'Incidence_Rate',
 'Case-Fatality_Ratio',
 'Unnamed: 0.1']


AttributeError: ignored

## Cleaning the dataset

There are Nan values that when perfoming statistics can raise exceptions. To 
avoid that behaviour, all the no valid values wil be converted to just a 0 constant.


In [45]:
# Is there any column with at least one NaN value?
df.isnull().any()

Province/State         True
Country/Region         True
Last Update            True
Confirmed              True
Deaths                 True
Recovered              True
Latitude               True
Longitude              True
FIPS                   True
Admin2                 True
Province_State         True
Country_Region         True
Last_Update            True
Lat                    True
Long_                  True
Active                 True
Combined_Key           True
Incidence_Rate         True
Case-Fatality_Ratio    True
dtype: bool

In [57]:
# To get the columns that has valid values, mask the dataframe with
# the column(s) of your like. Here I'll display the valid values
# each field has

table = []
header = ["Column", "Valid fiels"]

valid = df.notnull()
keys = list(df.keys())

for key in keys:
  valid_rows = df[valid[key]][key].count()
  table.append([key, valid_rows])

print(tabulate.tabulate(table, header))

Column                 Valid fiels
-------------------  -------------
Province/State                4358
Country/Region                7617
Last Update                   7617
Confirmed                   295836
Deaths                      295414
Recovered                   295467
Latitude                      4799
Longitude                     4799
FIPS                        255236
Admin2                      255894
Province_State              272637
Country_Region              288238
Last_Update                 288238
Lat                         283146
Long_                       283146
Active                      288238
Combined_Key                288238
Incidence_Rate               71952
Case-Fatality_Ratio          72305


In [87]:
# For this analisys I'm only interested in keeping the rows that have
# valid deaths, confirmed, recovered and Provice/State with Country/Region
# A mask can be used for multiple columns

columns = [
  "Province/State",
  "Country/Region",
  "Confirmed",
  "Deaths",
  "Recovered",
  "Last Update"
]

mask = valid[columns].apply(all, axis=1)
validDF = df[mask]
validDF.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
13,Hubei,Mainland China,1/22/2020 17:00,444.0,17.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Hubei,Mainland China,1/23/20 17:00,444.0,17.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Hubei,Mainland China,1/24/20 17:00,549.0,24.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Hubei,Mainland China,1/25/20 17:00,761.0,40.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Hubei,Mainland China,1/26/20 16:00,1058.0,52.0,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# There are columns that I won't need so it's better to drop them 
# This time the dataset is cleaned and I'll store it as csv file
# Also, the index will be reseted 

validDF = validDF.drop(columns=list(set(df.keys()) - set(columns))).reset_index(drop=True)

validDF.to_csv(dataset_path + "/" + "cleaned_total_cases.csv")

validDF.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,1/22/2020 17:00,444.0,17.0,28.0
1,Hubei,Mainland China,1/23/20 17:00,444.0,17.0,28.0
2,Hubei,Mainland China,1/24/20 17:00,549.0,24.0,31.0
3,Hubei,Mainland China,1/25/20 17:00,761.0,40.0,32.0
4,Hubei,Mainland China,1/26/20 16:00,1058.0,52.0,42.0


## Performing analisys 


Statistics can be computed easily with the built in methods pandas has for the Data Frame objects.

For this dataset, the fields of interest (numeric ones) are:
  + Confirmed 
  + Deaths
  + Recovered

In [92]:
# A general overview of the entire dataframe
# By default only numeric indices are included
# but if include="all" then also the non numeric
# ones are included

validDF.describe(include="all")

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
count,3997,3997,3997,3997.000000,3997.000000,3997.000000
unique,281,21,1270,NaN,NaN,NaN
top,Hubei,US,2020-03-11T20:00:00,NaN,NaN,NaN
freq,60,1530,80,NaN,NaN,NaN
mean,NaN,NaN,NaN,903.485864,29.513635,412.348511
std,NaN,NaN,NaN,6448.974269,262.668126,3662.431470
min,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,2.000000,0.000000,0.000000
50%,NaN,NaN,NaN,31.000000,0.000000,1.000000
75%,NaN,NaN,NaN,236.000000,2.000000,73.000000


In [98]:
# Quantiles: ratio of the data that's below a given threshold

validDF.quantile(q=np.arange(0.1, 1, 0.1))

,Confirmed,Deaths,Recovered
0.1,1.0,0.0,0.0
0.2,2.0,0.0,0.0
0.3,3.0,0.0,0.0
0.4,10.0,0.0,0.0
0.5,31.0,0.0,1.0
0.6,78.0,0.0,4.6
0.7,146.0,1.0,36.0
0.8,318.0,2.0,116.0
0.9,756.0,6.0,338.4


In [105]:
# Reducer sum to get the total number of confirmed, deaths and recovered cases

validDF.sum(numeric_only=True)

Confirmed    3611233.0
Deaths        117966.0
Recovered    1648157.0
dtype: float64

## The power of grouping Data Frames

+ Group by country and count
  + Confirmed 
  + Deaths
  + Recovered

+ Day with the highest numbers for each field

+ Country with the highest number for each field

In [132]:
# Grouping by Province/State and get the max number of cases

validDF.groupby("Province/State").max()

,Country/Region,Last Update,Confirmed,Deaths,Recovered
Province/State,,,,,
"Montreal, QC",Canada,2020-03-08T16:03:05,4.0,0.0,0.0
"Norfolk County, MA",US,2020-03-06T18:33:03,2.0,0.0,0.0
Alabama,US,2020-03-21T23:13:18,131.0,0.0,0.0
"Alameda County, CA",US,2020-03-07T13:13:14,2.0,0.0,0.0
Alaska,US,2020-03-21T23:13:18,15.0,0.0,0.0
...,...,...,...,...,...
Wyoming,US,2020-03-21T19:43:03,23.0,0.0,0.0
Xinjiang,Mainland China,2020-03-11T02:18:14,76.0,3.0,73.0
"Yolo County, CA",US,2020-03-06T20:13:14,1.0,0.0,0.0


In [133]:
validDF.groupby("Country/Region").max()

,Province/State,Last Update,Confirmed,Deaths,Recovered
Country/Region,,,,,
Australia,Western Australia,2020-03-21T13:13:35,436.0,6.0,8.0
Austria,None,2020-02-25T18:55:32,2.0,0.0,0.0
Canada,"Toronto, ON",2020-03-21T21:13:30,424.0,10.0,6.0
China,Zhejiang,2020-03-21T22:13:24,67800.0,3139.0,58946.0
Cruise Ship,Diamond Princess,2020-03-21T02:13:32,712.0,8.0,325.0
Denmark,Greenland,2020-03-21T12:43:08,1326.0,13.0,1.0
France,St Martin,2020-03-21T20:43:02,14282.0,562.0,12.0
Hong Kong,Hong Kong,2020-03-09T10:03:17,115.0,3.0,59.0
Hong Kong SAR,Hong Kong,2020-03-10T23:53:02,120.0,3.0,65.0


In [135]:
validDF.groupby("Deaths").max().tail()

,Province/State,Country/Region,Last Update,Confirmed,Recovered
Deaths,,,,,
3111.0,Hubei,China,2020-03-17T11:53:10,67799.0,56003.0
3122.0,Hubei,China,2020-03-18T12:13:09,67800.0,56927.0
3130.0,Hubei,China,2020-03-19T10:13:14,67800.0,57682.0
3133.0,Hubei,China,2020-03-20T07:43:02,67800.0,58382.0
3139.0,Hubei,China,2020-03-21T10:13:08,67800.0,58946.0
